<a href="https://colab.research.google.com/github/Trip1ewhy/Duke_xAI_Wayne/blob/main/Final_Project_Run_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set Up

In [1]:
!rm -r ./sample_data


In [2]:
# Please use this to connect your GitHub repository to your Google Colab notebook
# Connects to any needed files from GitHub and Google Drive
import os

# Remove Colab default sample_data


# Clone GitHub files to colab workspace
repo_name = "Duke_xAI_Wayne" # Change to your repo name
git_path = 'https://github.com/Trip1ewhy/Duke_xAI_Wayne.git' #Change to your path
!git clone "{git_path}"

# Install dependencies from requirements.txt file
#!pip install -r "{os.path.join(repo_name,'requirements.txt')}" #Add if using requirements.txt

# Change working directory to location of notebook
notebook_dir = 'Run-LLM'
path_to_notebook = os.path.join(repo_name,notebook_dir)
%cd "{path_to_notebook}"
%ls

Cloning into 'Duke_xAI_Wayne'...
remote: Enumerating objects: 210, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 210 (delta 33), reused 4 (delta 4), pack-reused 135 (from 1)
Receiving objects: 100% (210/210), 90.22 MiB | 28.46 MiB/s, done.
Resolving deltas: 100% (70/70), done.
[Errno 2] No such file or directory: 'Duke_xAI_Wayne/Run-LLM'
/content
Duke_xAI_Wayne/


In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `Wayne_duke_xAI` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authen

In [4]:

pip install markdown sentence-transformers transformers faiss-cpu gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.2/168.2 kB 11.1 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
import os
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer
import faiss
import numpy as np
import torch
import gradio as gr

# Initialize retrieval and generation models
def initialize_models():
    # Retrieval model
    retriever_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2").to("cuda")
    # Generation model
    generator_model_name = "meta-llama/Llama-3.1-8B-Instruct"
    generator_tokenizer = AutoTokenizer.from_pretrained(generator_model_name)
    generator_model = AutoModelForCausalLM.from_pretrained(generator_model_name, device_map="auto", torch_dtype=torch.float16)
    return retriever_model, generator_tokenizer, generator_model

# Load documents from an external TXT file
def load_documents_from_txt(file_path):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"The file {file_path} does not exist.")

    documents = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            if line:  # Skip empty lines
                documents.append({"content": line})
    return documents

# Build the knowledge base index
def build_index(documents, retriever_model):
    embeddings = retriever_model.encode([doc['content'] for doc in documents], convert_to_tensor=True, device="cuda")
    embeddings = embeddings.cpu().numpy()
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(np.array(embeddings))
    return index, embeddings

# Retrieve relevant documents
def retrieve(query, index, retriever_model, documents, top_k=3):
    query_embedding = retriever_model.encode(query, convert_to_tensor=True, device="cuda").cpu().numpy()
    distances, indices = index.search(query_embedding.reshape(1, -1), top_k)
    retrieved_docs = [documents[idx] for idx in indices[0]]
    return retrieved_docs

# Generate an answer using the generation model
# Generate an answer using the generation model
def generate_answer(query, retrieved_docs, generator_model, generator_tokenizer):
    # Define a system prompt
    system_prompt = (
        "You are an expert in the field of human morality and safety. You will be asked a series of questions related to human morality and safety, and you should respond in the following format: Reasoning: *****. // This is your thought process regarding the question. Response: *****. // This is your answer to the user."
    )
    # Combine the system prompt with retrieved context
    context = " ".join([doc['content'] for doc in retrieved_docs])
    input_text = f"{system_prompt} \n\n Question: {query}\n  Context: {context}\n   Answer:"

    # Tokenize and generate output
    inputs = generator_tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).to("cuda")
    outputs = generator_model.generate(
        inputs["input_ids"],
        max_length=150,
        num_beams=3,
        early_stopping=True
    )
    answer = generator_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

# Load models and knowledge base
retriever_model, generator_tokenizer, generator_model = initialize_models()
txt_file_path = "/content/Duke_xAI_Wayne/moral LLM - project/USCODE-2010-title20-chap31-subchapIII-part4-sec1232h copy.txt"  # Replace with your file path
documents = load_documents_from_txt(txt_file_path)
index, embeddings = build_index(documents, retriever_model)

# Define Gradio interface
def chat_with_rag(user_query):
    retrieved_docs = retrieve(user_query, index, retriever_model, documents)
    answer = generate_answer(user_query, retrieved_docs, generator_model, generator_tokenizer)
    retrieved_texts = "\n".join([doc['content'] for doc in retrieved_docs])
    return f"Retrieved Context:\n\n{retrieved_texts}\n\n\n\n\nAnswer:\n\n{answer}"

# Create Gradio interface
interface = gr.Interface(
    fn=chat_with_rag,
    inputs=gr.Textbox(label="Enter your question:"),
    outputs=gr.Textbox(label="Chatbot Response"),
    title="MORAL LLM Chatbot"
)

# Launch Gradio app
interface.launch(share=True)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]